In [ ]:
import torch
from torchvision import transforms, models
from captum.attr import IntegratedGradients, LayerGradCam
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load the pre-trained model (Assuming it's a ResNet or similar CNN model)
model = models.resnet18(pretrained=True)
model.eval()

# Preprocessing function for input images
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess an example image
def process_image(image_path):
    image = Image.open(image_path)
    input_tensor = preprocess(image).unsqueeze(0)
    return input_tensor

# Replace 'path_to_image' with the actual path of the test image
input_image = process_image('path_to_image')


#### Integrated Gradients
Integrated Gradients is an attribution method designed to highlight the important features of the input image that contribute to the model's classification. It works by integrating gradients along the path from a baseline (e.g., black image) to the input image. This method is particularly useful for visualizing how much each pixel contributes to the model's decision.


In [ ]:
# Initialize Integrated Gradients with the model
ig = IntegratedGradients(model)

# Compute attributions
attributions, delta = ig.attribute(input_image, target=0, baselines=input_image * 0, return_convergence_delta=True)

# Visualize the attributions
_ = viz.visualize_image_attr(attributions.squeeze().cpu().detach().numpy().transpose(1, 2, 0),
                             method="heat_map", sign="all", show_colorbar=True, title="Integrated Gradients")


#### Grad-CAM
Grad-CAM (Gradient-weighted Class Activation Mapping) is another method to visualize a model's attention areas by back-propagating gradients and computing their influence on certain convolutional layers. This is useful for identifying spatial features in CNN models and visualizing which parts of the image are important for a particular class.


In [ ]:
# Select the target layer in the model (usually the last convolutional layer)
target_layer = model.layer4[1].conv2

# Initialize Grad-CAM with the model and target layer
grad_cam = LayerGradCam(model, target_layer)

# Compute Grad-CAM attributions
attributions = grad_cam.attribute(input_image, target=0)

# Upsample and visualize the Grad-CAM heatmap
_ = viz.visualize_image_attr(attributions[0].cpu().detach().numpy(),
                             method="blended_heat_map", sign="all", show_colorbar=True, title="Grad-CAM")


V### Analysis
From the above visualizations:
1. **Integrated Gradients** provides a pixel-level attribution, where we can observe fine-grained details about the areas contributing to the classification.
2. **Grad-CAM** produces a more generalized heatmap, highlighting spatial regions that the model focused on, which can help identify overall regions of interest in the image.

These two methods complement each other: Integrated Gradients gives a detailed pixel-level insight, while Grad-CAM shows broader regions of attention.
